In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import load, Loader
import subprocess
from json import dump

## Parameters for Processing

In [14]:
output_path = Path("run2")
actual_path = output_path / "actual_time.yaml"
theoretical_path = output_path / "theoretical_time.yaml"
metadata_path = output_path / "metadata.yaml"
mem_folder_path = output_path / "memory_output"

data_dump_path = Path("processed_output")
output_fnames = ("actual_runtime.csv", "theoretical_runtime.csv", "bytes.csv", "all_fields.json")
MFLOPS = 870
FLOPS = MFLOPS * 10E6
MODELS = ["tf-necd", "tf-cod", "pytorch-qrcp", "pytorch-qr", "pytorch-svd", "pytorch-svddc", "sklearn-svddc"]

## Loading Metadata

In [15]:
with open(metadata_path, "r") as f:
    metadata = load(f, Loader=Loader)
    
metadata

{'dataset_shape': '1000000000 x 10',
 'failed_regs': [],
 'failed_regs_exceptions': [],
 'repeat': 2,
 'rows_in_experiment': [10,
  31,
  100,
  316,
  1000,
  3162,
  10000,
  31622,
  100000,
  316227,
  1000000,
  3162277,
  10000000,
  31622776,
  100000000,
  316227766,
  1000000000],
 'timer_method': 'process in nanoseconds'}

## Loading Actual Times

In [16]:
with open(actual_path, "r") as f:
    actual_times = load(f, Loader=Loader)
    
actual_times_dict = {reg_name: {row_count: np.mean([nanoseconds for rc, nanoseconds in actual_times if row_count == rc]) for row_count, _ in actual_times} for reg_name, actual_times in actual_times.items()}
actual_times_dict

{'pytorch-qr': {10: 685287.5,
  31: 430181.0,
  100: 373516.0,
  316: 390501.0,
  1000: 398536.0,
  3162: 445676.5,
  10000: 639281.5,
  31622: 1471663.5,
  100000: 3783824.0,
  316227: 12640497.0,
  1000000: 60955017.0,
  3162277: 210996533.0,
  10000000: 705990544.5,
  31622776: 2565892599.5,
  100000000: 9023135835.5,
  316227766: 28464308157.5,
  1000000000: 92707393493.0},
 'pytorch-qrcp': {10: 20033590.0,
  31: 485327.5,
  100: 560836.0,
  316: 519296.5,
  1000: 389220.0,
  3162: 455876.0,
  10000: 751162.0,
  31622: 1695440.0,
  100000: 4465915.0,
  316227: 14067657.0,
  1000000: 63800027.5,
  3162277: 225718777.5,
  10000000: 774098713.0,
  31622776: 2720046574.0,
  100000000: 9466190326.5,
  316227766: 29387135167.0,
  1000000000: 97388130946.0},
 'pytorch-svd': {10: 1235908.5,
  31: 428611.0,
  100: 417036.0,
  316: 441661.0,
  1000: 441516.5,
  3162: 492550.5,
  10000: 708556.5,
  31622: 1546504.0,
  100000: 3817597.5,
  316227: 12474244.5,
  1000000: 60972131.0,
  3162277: 

## Loading Theoretical Flop Counts

In [17]:
with open(theoretical_path, "r") as f:
    theoretical_flops = load(f, Loader=Loader)
    
theoretical_times_dict = {reg_name: {row_count: (flops / FLOPS) * 10E9 for row_count, flops in theoretical_flops} for reg_name, theoretical_flops in theoretical_flops.items()}
# theoretical_times_dict

## Loading / Processing Binary Memory Files

In [18]:
make_mempath = lambda reg_name, row_count, iter, ext: mem_folder_path / f"{'mem' if ext == 'bin' else 'memray-csv-mem'}_{reg_name}_{row_count}_{iter}.{ext}"

mem_dict = {}
for reg_name in MODELS:
    mem_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        runs = []
        for iter in range(metadata["repeat"]):
            # use memray transform to actually create csv
            input_path = make_mempath(reg_name, row_count, iter, "bin")
            print(f"{input_path =}")
            subprocess.run(["memray", "transform", "csv", input_path, "-f"])
            
            # load csv normally, sum all mem usage
            output_path = make_mempath(reg_name, row_count, iter, "csv")
            single_mem_df = pd.read_csv(output_path, header=0, index_col=None)
            bytes_used = single_mem_df["size"].sum()
            runs.append(bytes_used)
            
        mem_dict[reg_name][row_count] = np.mean(runs)

input_path =PosixPath('run2/memory_output/mem_tf-necd_10_0.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote run2/memory_output/memray-csv-mem_tf-necd_10_0.csv
input_path =PosixPath('run2/memory_output/mem_tf-necd_10_1.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote run2/memory_output/memray-csv-mem_tf-necd_10_1.csv
input_path =PosixPath('run2/memory_outpu

## Aggregate and Output Data

In [19]:
json_prep_dict = {}
for reg_name in MODELS:
    json_prep_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        json_prep_dict[reg_name][row_count] = {
            "real_runtime": actual_times_dict[reg_name][row_count],
            "theoretical_runtime": theoretical_times_dict[reg_name][row_count],
            "bytes": mem_dict[reg_name][row_count]
        }

df_real_runtime = pd.DataFrame(actual_times_dict)
df_theoretical_runtime = pd.DataFrame(theoretical_times_dict)
df_memory = pd.DataFrame(mem_dict)

data_dump_path.mkdir(exist_ok=True)

df_real_runtime.to_csv(data_dump_path / output_fnames[0], header=True, index=True)
df_theoretical_runtime.to_csv(data_dump_path / output_fnames[1], header=True, index=True)
df_memory.to_csv(data_dump_path / output_fnames[2], header=True, index=True)

with open(data_dump_path / output_fnames[3], "w") as f:
    dump(json_prep_dict, f)
    
print(f"All data has been dumped to: {data_dump_path}")

All data has been dumped to: processed_output
